## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-24-18-51-56 +0000,wapo,Trump cancels trade negotiations with Canada o...,https://www.washingtonpost.com/world/2025/10/2...
1,2025-10-24-18-48-59 +0000,bbc,Chief prosecutor says top official would not c...,https://www.bbc.com/news/articles/clygwjp8yzzo...
2,2025-10-24-18-45-34 +0000,nyt,Trump and Canada’s Trade Spat Spotlights Doug ...,https://www.nytimes.com/2025/10/24/world/canad...
3,2025-10-24-18-45-10 +0000,bbc,Woman gets rare whole-life sentence for murder...,https://www.bbc.com/news/articles/cjr07wqeerwo...
4,2025-10-24-18-43-00 +0000,wsj,"Procter & Gamble to Focus on Innovation, Not D...",https://www.wsj.com/business/retail/procter-ga...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2380/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,57
182,will,20
39,new,18
85,off,15
4,canada,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
257,2025-10-24-03-21-49 +0000,cbc,Trump claims Canada 'CHEATED AND GOT CAUGHT' w...,https://www.cbc.ca/news/world/trump-trade-nego...,141
258,2025-10-24-03-16-11 +0000,nypost,Trump terminates trade talks with Canada over ...,https://nypost.com/2025/10/23/us-news/trump-te...,128
297,2025-10-23-23-29-06 +0000,nypost,Ford warns devasting plant fire will hurt prof...,https://nypost.com/2025/10/23/business/ford-wa...,127
244,2025-10-24-05-00-38 +0000,bbc,"Trump will meet Xi in South Korea, says White ...",https://www.bbc.com/news/articles/cjw9vy19qvgo...,113
93,2025-10-24-15-50-54 +0000,nyt,Trump Says He’s Cutting Off Trade Negotiations...,https://www.nytimes.com/2025/10/23/us/politics...,113


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
257,141,2025-10-24-03-21-49 +0000,cbc,Trump claims Canada 'CHEATED AND GOT CAUGHT' w...,https://www.cbc.ca/news/world/trump-trade-nego...
101,72,2025-10-24-15-22-00 +0000,wsj,Annual inflation heated up slightly in Septemb...,https://www.wsj.com/economy/consumer-price-ind...
297,70,2025-10-23-23-29-06 +0000,nypost,Ford warns devasting plant fire will hurt prof...,https://nypost.com/2025/10/23/business/ford-wa...
111,55,2025-10-24-14-59-00 +0000,wsj,The extended government shutdown has put colle...,https://www.wsj.com/economy/shutdown-puts-octo...
356,53,2025-10-23-20-01-18 +0000,nypost,The corporations and donors footing the bill f...,https://nypost.com/2025/10/23/us-news/here-are...
135,46,2025-10-24-13-54-28 +0000,nypost,"‘Sex warfare’: China, Russia send hot women to...",https://nypost.com/2025/10/24/business/china-r...
153,44,2025-10-24-13-00-06 +0000,wapo,Rubio stresses there is ‘no plan B’ amid quest...,https://www.washingtonpost.com/world/2025/10/2...
68,37,2025-10-24-16-45-36 +0000,nypost,Company behind ladder truck used in daring Lou...,https://nypost.com/2025/10/24/us-news/company-...
104,37,2025-10-24-15-15-04 +0000,nypost,Letitia James mimics MLK’s ‘I Have A Dream’ sp...,https://nypost.com/2025/10/24/us-news/new-york...
37,36,2025-10-24-17-51-50 +0000,nypost,US deploys Ford carrier strike group to target...,https://nypost.com/2025/10/24/us-news/us-deplo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
